- `review_date` and `claim_date` is not always yyyy-mm-dd
    - Contains yyyy-mm-ddThh:mm:ss and None

In [2]:
from helper import download_dataset, download_article

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
DATASET_FP = "./WatClaimCheck_dataset" # CHANGE TO MATCH LOCAL
train_pd_df, valid_pd_df, test_pd_df = download_dataset(DATASET_FP)

In [4]:
def explode_dictionary(pd_df: pd.DataFrame, field: str) -> pd.DataFrame:
    '''Explodes a dictionary within a column as multiple columns and then drops the parent field'''
    return pd.concat([pd_df.drop(field, axis=1), pd_df[field].apply(pd.Series)], axis=1)

def clean_pd_df(pd_df: pd.DataFrame) -> pd.DataFrame:
    '''Extracts field from dataframe and removes id'''
    print("Extracting fields from metadata")
    pd_df = explode_dictionary(pd_df, 'metadata')
    print("Extracting fields from label")
    pd_df = explode_dictionary(pd_df, 'label')

    pd_df = pd_df.drop(columns=['id'])
    
    return pd_df

In [5]:
flatten_train_pd_df = clean_pd_df(train_pd_df)
flatten_valid_pd_df = clean_pd_df(valid_pd_df)
flatten_test_pd_df = clean_pd_df(test_pd_df)

Extracting fields from metadata
Extracting fields from label
Extracting fields from metadata
Extracting fields from label
Extracting fields from metadata
Extracting fields from label


In [6]:
flatten_test_pd_df

,claimant,claim,claim_date,review_date,premise_articles,reviewer_name,reviewer_site,review_url,rating,original_rating,review_article
0,None,President Trump played golf six times during h...,None,2017-02-23,{'https://twitter.com/realDonaldTrump/status/5...,Snopes,snopes.com,https://www.snopes.com/fact-check/trump-obama-...,1,mostly true,30349.json
1,Multiple sources,Buhari dead,2020-10-18,2020-10-23,{'http://perma.cc/HL3H-H6VY?type=image': '3035...,AFP Fact Check,factcheck.afp.com,https://factcheck.afp.com/rumours-nigerian-lea...,0,false,30350.json
2,Donald Trump,Claims other countries abuse the Diversity Imm...,2018-10-22,2018-10-25,{'https://www.facebook.com/sharer/sharer.php?u...,FactCheck.org,factcheck.org,https://www.factcheck.org/2018/10/trump-stump-...,0,false,30351.json
3,None,The top 10 Commonwealth countries take 8% of U...,None,2017-06-02,{'http://www.bbc.co.uk/iplayer/episode/b08sh4g...,Full Fact,fullfact.org,https://fullfact.org/economy/uk-trade-commonwe...,2,correct.,30352.json
4,None,"A group calling itself ""Antifa America"" threat...",None,2020-06-02,{'https://web.archive.org/web/20200602122049/h...,Snopes,snopes.com,https://www.snopes.com/fact-check/antifa-hoax-...,0,false,30353.json
...,...,...,...,...,...,...,...,...,...,...,...
3368,FB page 'I support Ravish Kumar',Farmer beaten up by Rajasthan police,2016-06-20,2018-06-06,{'https://www.facebook.com/ISupportRavishNDTVi...,Alt News,altnews.in,https://www.altnews.in/image-of-rape-accused-f...,0,false,33717.json
3369,Richard Corcoran,"""Spending more taxpayer money on VISIT FL (or ...",2017-02-06,2017-02-09,{'http://www.tampabay.com/news/politics/stater...,PolitiFact,politifact.com,https://www.politifact.com/factchecks/2017/feb...,1,half-true,33718.json
3370,Mick Mulvaney,"""They voted for (a border wall) in 2006. Then-...",2017-04-23,2017-04-23,{'https://www.politifact.com/truth-o-meter/art...,PolitiFact,politifact.com,https://www.politifact.com/factchecks/2017/apr...,1,half-true,33719.json
3371,Wes Riddle,Says combined inflation and unemployment rates...,2011-10-22,2012-06-08,"{'http://www.miseryindex.us/': '33720_1.json',...",PolitiFact,politifact.com,https://www.politifact.com/factchecks/2012/jun...,0,false,33720.json
